In [32]:
# Import packages
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime

In [33]:

headers = {
    'authority': 'www.amazon.com',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-language': 'en-US,en;q=0.9,bn;q=0.8',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="102", "Google Chrome";v="102"',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'
}

In [34]:
reviews_url = 'https://www.amazon.com/Legendary-Whitetails-Journeyman-Jacket-Tarmac/product-reviews/B013KW38RQ/'

In [35]:

len_page = 1000

In [36]:

def reviewsHtml(url, len_page):
    soups = []
    for page_no in range(1, len_page + 1):
        params = {
            'ie': 'UTF8',
            'reviewerType': 'all_reviews',
            'filterByStar': 'critical',
            'pageNumber': page_no,
        }
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'lxml')
        soups.append(soup)
    return soups

In [37]:

def getReviews(html_data):
    data_dicts = []
    boxes = html_data.select('div[data-hook="review"]')
    for box in boxes:
        try:
            name = box.select_one('[class="a-profile-name"]').text.strip()
        except Exception as e:
            name = 'N/A'
        try:
            stars = box.select_one('[data-hook="review-star-rating"]').text.strip().split(' out')[0]
        except Exception as e:
            stars = 'N/A'   
        try:
            title = box.select_one('[data-hook="review-title"]').text.strip()
        except Exception as e:
            title = 'N/A'
        try:
            datetime_str = box.select_one('[data-hook="review-date"]').text.strip().split(' on ')[-1]
            date = datetime.strptime(datetime_str, '%B %d, %Y').strftime("%d/%m/%Y")
        except Exception as e:
            date = 'N/A'
        try:
            description = box.select_one('[data-hook="review-body"]').text.strip()
        except Exception as e:
            description = 'N/A'
        data_dict = {
            'Name' : name,
            'Stars' : stars,
            'Title' : title,
            'Date' : date,
            'Description' : description
        }
        data_dicts.append(data_dict)
    return data_dicts

In [38]:

html_datas = reviewsHtml(reviews_url, len_page)

In [27]:

reviews = []

In [28]:

for html_data in html_datas:
    review = getReviews(html_data)
    reviews += review

In [29]:

df_reviews = pd.DataFrame(reviews)

In [30]:
df_reviews

,Name,Stars,Title,Date,Description
0,Erica,5.0,"5.0 out of 5 stars\nQuality. But, the sizes ar...",18/12/2023,"I am a large, sometimes an XL. I ordered a 4XL..."
1,Melissa Sugar,5.0,"5.0 out of 5 stars\nDurable, Attractive, Multi...",01/08/2022,I purchased various Legendary Whitetails Men's...
2,Anthony L Leslie Jr.,5.0,5.0 out of 5 stars\nPerfect fit,28/01/2024,"I love this jacket shirt. It is really nice, w..."
3,Jeff,4.0,"4.0 out of 5 stars\nRugged, good look and nice...",02/10/2018,The short answer to if you should go down from...
4,Amazon Customer,5.0,5.0 out of 5 stars\nGreat Purchase!,18/01/2024,I got this for my husband for Christmas. Appar...
...,...,...,...,...,...
2995,Brenda,5.0,5.0 out of 5 stars\nGreat quality and good fit,15/01/2024,Got this for my husband not knowing if it was ...
2996,A. Westwood,3.0,"3.0 out of 5 stars\nGreat fabrics and sewing, ...",19/01/2024,"Great fabric and lining, this ""Shaket"" is made..."
2997,Amazon Customer,5.0,5.0 out of 5 stars\nMy son loved it,05/02/2024,This is a really nice jacket. It’s lined and ...
2998,Jeff E,5.0,5.0 out of 5 stars\nNew favorite piece,08/02/2024,"Was looking for a ""sh-acket"" that was not only..."


In [31]:

df_reviews.to_csv('reviews.csv', index=False)